In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import math

In [ ]:
# entropy 
def entropy(c, n):
    return -(c*1.0/n)*math.log(c*1.0/n, 2)
    